In [1]:
import pandas as pd
import math

In [2]:

data={
    "Weather":['Sunny' ,'Strong Wind', 'Partly Cloudy', 'Cloudy']
        ,
     "y":["Yes", "Yes", "Yes", 'Yes']
}
dataset=pd.DataFrame(
data
)

In [3]:
z=dataset['y']

In [4]:
z[z=='Yes'].count()

np.int64(4)

In [5]:
for i in z.unique():
    print(i)

Yes


In [6]:
def entropy(feature):
    entropy = 0
    for i in feature.unique():
        p_i=feature[feature==i].count()/len(feature)
        entropy += -1*p_i * math.log2(p_i)
    return entropy

In [7]:
entropy(dataset['y'])

np.float64(0.0)

In [8]:
def information_gain(feature,dataset):
    info_d = entropy(dataset['y'])
    info_a=0
    for i in feature.unique():
        d = dataset[feature==i]
        info_a+= (len(d)/len(dataset))*entropy(d['y'])
    gain = info_d - info_a
    return gain


In [9]:
import pandas as pd

data = [
    ("<=30", "high", "no", "fair", "no"),
    ("<=30", "high", "no", "excellent", "no"),
    ("31...40", "high", "no", "fair", "yes"),
    (">40", "medium", "no", "fair", "yes"),
    (">40", "low", "yes", "fair", "yes"),
    (">40", "low", "yes", "excellent", "no"),
    ("31...40", "low", "yes", "excellent", "yes"),
    ("<=30", "medium", "no", "fair", "no"),
    ("<=30", "low", "yes", "fair", "yes"),
    (">40", "medium", "yes", "fair", "yes"),
    ("<=30", "medium", "yes", "excellent", "yes"),
    ("31...40", "medium", "no", "excellent", "yes"),
    ("31...40", "high", "yes", "fair", "yes"),
    (">40", "medium", "no", "excellent", "no"),
]

columns = ["age", "income", "student", "credit_rating", "y"]
df = pd.DataFrame(data, columns=columns)

print(df)


        age  income student credit_rating    y
0      <=30    high      no          fair   no
1      <=30    high      no     excellent   no
2   31...40    high      no          fair  yes
3       >40  medium      no          fair  yes
4       >40     low     yes          fair  yes
5       >40     low     yes     excellent   no
6   31...40     low     yes     excellent  yes
7      <=30  medium      no          fair   no
8      <=30     low     yes          fair  yes
9       >40  medium     yes          fair  yes
10     <=30  medium     yes     excellent  yes
11  31...40  medium      no     excellent  yes
12  31...40    high     yes          fair  yes
13      >40  medium      no     excellent   no


In [10]:
information_gain(df["age"],df)

np.float64(0.24674981977443933)

In [11]:
information_gain(df["income"],df)

np.float64(0.02922256565895487)

In [12]:
information_gain(df["credit_rating"],df)


np.float64(0.04812703040826949)

In [13]:
information_gain(df["student"],df)


np.float64(0.15183550136234159)

In [14]:
def best_split_feature(dataset):
    X = dataset.drop(['y'],axis=1)
    best_index = -1
    best_gain=0
    for feature in X.columns:
        gain = information_gain(dataset[feature],dataset)
        if(gain>best_gain):
            best_gain = gain
            best_index = feature
    return best_index

In [15]:
best_split_feature(df)

'age'

In [16]:
class Node :
    def __init__(self,value,name):
        self.value = value
        self.name=name
        self.children= []
    def _is_leaf(self):
        return len(self.children)==0


In [17]:
df.shape

(14, 5)

In [18]:
##train
def build_tree(dataset,parent_node=None):
    if df.shape[1]<=1:
        leaf_value = dataset['y'].mode()[0]
        return Node(leaf_value,f"Leaf:{leaf_value}")
    best_feature=best_split_feature(dataset)
    if best_feature==-1:
        leaf_value = dataset['y'].mode()[0]
        return Node(leaf_value,f"Leaf:{leaf_value}")
    node = Node(best_feature,f"Feature : {best_feature}")
    for values in dataset[best_feature].unique():
        sub_dataset=dataset[dataset[best_feature]==values]
        child_node=build_tree(sub_dataset,node)
        node.children.append((values,child_node))
    return node


In [19]:
def print_tree(node,level=0):
    indent=' '*level
    print(f"{indent}{node.name}")
    for value,child in node.children:
        print(f"{indent}|--{value}")
        print_tree(child,level+1)

In [20]:
root=build_tree(df)

In [21]:
print_tree(root)

Feature : age
|--<=30
 Feature : student
 |--no
  Leaf:no
 |--yes
  Leaf:yes
|--31...40
 Leaf:yes
|-->40
 Feature : credit_rating
 |--fair
  Leaf:yes
 |--excellent
  Leaf:no


In [22]:
def predict(root:Node,X:pd.DataFrame,index):
    if root._is_leaf() :
        return root.value
    val=str(X.loc[index,root.value])
    for value,child in root.children:
        if value.__contains__(val):
            return predict(child,X,index)
            break

In [23]:
X=df.iloc[3:4,:]
print(predict(root,X,3))


yes
